In [2]:
import transformers
import torch
from huggingface_hub import notebook_login
import os
from openai import AzureOpenAI
import ast 
from tqdm import tqdm 
import numpy as np
import json
import ot
notebook_login()
import sys
sys.path.append('../') 
from SAKE.distributions import *
from SAKE.threshold import *
from SAKE.edit import *

In [3]:
if torch.backends.mps.is_available(): device = torch.device("mps")
else: device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'Using device: {device}')

Using device: mps


In [4]:
import json

with open('../data/cf/example.json') as f:
    cf = json.load(f)

# with open('../data/cf/counterfact.json') as f:
#     cf = json.load(f)

indexes = (0,5) # change if you want to do other edits

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = 'gpt2-xl'
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states = True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:777: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [6]:
maps = []
for i in tqdm(range(indexes[0], indexes[1])):
    e = cf[i]
    x = np.concatenate([e['forced_source_embs'], e['source_embs'], e['target_embs']])
    x_source = x[:2*len(e['forced_source_embs'])]
    x_target = x[2*len(e['forced_source_embs']):]
    ot_linear = ot.da.LinearTransport(reg=1e-2)
    ot_linear.fit(Xs=x_source, Xt=x_target)
    maps.append(ot_linear)

100%|██████████| 5/5 [00:03<00:00,  1.66it/s]


In [ ]:
magnitude = []
prob_success = 0
dI_a = 0
dI_rev = 0
for i in tqdm(range(150)):
    # e = cf[i]
    # x = np.concatenate([e['forced_source_embs'], e['source_embs'], e['target_embs'], e['target_embs']])
    # x_source = x[:2*len(e['forced_source_embs'])]
    # x_target = x[2*len(e['forced_source_embs']):]
    # ot_linear = ot.da.LinearTransport(reg=1e-2)
    # ot_linear.fit(Xs=x_source, Xt=x_target)
    prompt = cf[i]['requested_rewrite']['prompt'].replace('{}', cf[i]['requested_rewrite']['subject'])
    doubt = prompt + " " + cf[i]['requested_rewrite']['target_new']['str'] + "??? No, it can't be, This contradicts my understanding. It is pretty clear that the answer to '" +  prompt + "' is something else. Rethink and give the correct answer:"
    input_ids = tokenizer(doubt, return_tensors="pt").to('mps').input_ids
    with torch.no_grad():
        model.eval()
        output = model(input_ids)

    last_h = output.hidden_states[-1][:,-1:,:].view(-1).cpu()
    pre_sim, max_index = hid_threshold(last_h, [source_mean_embs[i]], 1000, dist_type = "euc")

    if max_index is not None:
        # last_h = torch.tensor(ot_linear.transform(Xs=last_h), dtype = torch.float16)
        last_h += torch.tensor(mean_diffs[i], dtype = torch.float16)
        last_h = last_h.view(-1)
    else:
        last_h = output.hidden_states[-1][:,-1,:].cpu()
    logits = torch.matmul(last_h, model.lm_head.weight.T.cpu())
    next_token_probs = torch.softmax(logits, dim=-1)

    if next_token_probs.shape[0] == 1:
        next_token_probs = next_token_probs[0]
        
    new_object_token = cf[i]['requested_rewrite']['target_new']['str']
    old_object_token = cf[i]['requested_rewrite']['target_true']['str']

    next_token_id = torch.argmax(next_token_probs, dim = -1)
    next_token = tokenizer.decode(next_token_id).strip()

    if new_object_token.startswith(next_token) and next_token != "":
        dI_a += 1

    if old_object_token.startswith(next_token) and next_token != "":
        dI_rev += 1
        
    if i % 10 == 0:
        print(dI_a)